In [72]:
import requests
import os

API_KEY = os.getenv("API_KEY")

SPORT = 'soccer_epl' 
REGIONS = 'uk' 
MARKETS = 'h2h' 
ODDS_FORMAT = 'decimal' 
DATE_FORMAT = 'iso' 


In [73]:
odds_response = requests.get(
    f'https://api.the-odds-api.com/v4/sports/{SPORT}/odds',
    params={
        'api_key': API_KEY,
        'regions': REGIONS,
        'markets': MARKETS,
        'oddsFormat': ODDS_FORMAT,
        'dateFormat': DATE_FORMAT,
    }
)

if odds_response.status_code != 200:
    print(f'Failed to get odds: status_code {odds_response.status_code}, response body {odds_response.text}')

else:
    odds_json = odds_response.json()

    print('Remaining requests', odds_response.headers['x-requests-remaining'])
    print('Used requests', odds_response.headers['x-requests-used'])


Remaining requests 475
Used requests 25


In [74]:
if odds_response.status_code != 200:
    print(f'Failed to get odds: status_code {odds_response.status_code}, response body {odds_response.text}')

else:
    odds_json = odds_response.json()

    print('Remaining requests', odds_response.headers['x-requests-remaining'])
    print('Used requests', odds_response.headers['x-requests-used'])


Remaining requests 475
Used requests 25


In [75]:
import pandas as pd
from collections import defaultdict

df = pd.json_normalize(
    odds_json,
    record_path=['bookmakers','markets','outcomes'],
    meta=['id','sport_key','sport_title','commence_time','home_team','away_team',
            ['bookmakers','title'], ['bookmakers','last_update'],
            ['bookmakers','markets','last_update']],
    errors='ignore'
)

df = (df.rename(columns={
        'id':'event_id',
        'bookmakers.title':'bookmaker',
        'bookmakers.last_update':'bookmaker_last_update',
        'bookmakers.markets.last_update':'market_last_update',
        'name':'outcome_name',
        'price':'odds_decimal'
    })
    [['event_id','commence_time','home_team','away_team',
        'bookmaker','bookmaker_last_update',
        'outcome_name','odds_decimal']]
)

print(df.tail(10))


                             event_id         commence_time   home_team  \
908  65d0fd91ca519ad5fcc3bca2ecc34b2e  2025-11-03T20:00:00Z  Sunderland   
909  65d0fd91ca519ad5fcc3bca2ecc34b2e  2025-11-03T20:00:00Z  Sunderland   
910  65d0fd91ca519ad5fcc3bca2ecc34b2e  2025-11-03T20:00:00Z  Sunderland   
911  65d0fd91ca519ad5fcc3bca2ecc34b2e  2025-11-03T20:00:00Z  Sunderland   
912  65d0fd91ca519ad5fcc3bca2ecc34b2e  2025-11-03T20:00:00Z  Sunderland   
913  65d0fd91ca519ad5fcc3bca2ecc34b2e  2025-11-03T20:00:00Z  Sunderland   
914  65d0fd91ca519ad5fcc3bca2ecc34b2e  2025-11-03T20:00:00Z  Sunderland   
915  65d0fd91ca519ad5fcc3bca2ecc34b2e  2025-11-03T20:00:00Z  Sunderland   
916  65d0fd91ca519ad5fcc3bca2ecc34b2e  2025-11-03T20:00:00Z  Sunderland   
917  65d0fd91ca519ad5fcc3bca2ecc34b2e  2025-11-03T20:00:00Z  Sunderland   

    away_team      bookmaker bookmaker_last_update outcome_name  odds_decimal  
908   Everton    Unibet (UK)  2025-10-25T20:28:31Z         Draw          3.10  
909   Everton 

In [76]:
#{(home_team, away_team): {outcome: [(odds, bookmaker), ...]}}
odds_by_match = {}

for _, row in df.iterrows():
    match_key = (row['home_team'], row['away_team'])
    outcome = row['outcome_name']
    odds = row['odds_decimal']
    bookmaker = row['bookmaker']
    
    if match_key not in odds_by_match:
        odds_by_match[match_key] = {}
    
    if outcome not in odds_by_match[match_key]:
        odds_by_match[match_key][outcome] = []
    
    odds_by_match[match_key][outcome].append((odds, bookmaker))


In [78]:
import pprint

# accounts for slippage, ect
epsilon = 0.05

def calc_arbitrage(odds1, odds2, odds3):
    return (1/(odds1)) + (1/(odds2)) + (1/(odds3))

def find_arbritrage(odds_by_match):
    oppourtunities = []
    for match_key, outcomes in odds_by_match.items():

        if len(outcomes) != 3:
            continue

        best_h_val = 1 - epsilon
        current_best = None
    
        outcome1, outcome2, outcome3 = outcomes.keys()

        for odds1, bookie1 in outcomes[outcome1]:
            for odds2, bookie2 in outcomes[outcome2]:
                for odds3, bookie3 in outcomes[outcome3]:

                    if len({bookie1, bookie2, bookie3}) == 3:
                        h_val = calc_arbitrage(odds1, odds2, odds3)
                        if h_val < best_h_val:
                            best_h_val = h_val
                            current_best = {
                                outcome1: (odds1, bookie1),
                                outcome2: (odds2, bookie2),
                                outcome3: (odds3, bookie3)
                            }

        if current_best:
            profit_margin = ((1/best_h_val)-1)*100
            oppourtunities.append({
                'match': f"{match_key[0]} vs {match_key[1]}",
                'h_val': best_h_val,
                'profit_margin_%': profit_margin,
                'combination': current_best
            })
    
    return sorted(oppourtunities, key = lambda x : x["h_val"])
    
arb_opps = find_arbritrage(odds_by_match)
pprint.pprint(arb_opps)


[{'combination': {'Brentford': (1.95, 'Bet Victor'),
                  'Draw': (6.8, 'Betfair'),
                  'Liverpool': (20.0, 'Matchbook')},
  'h_val': 0.7098793363499247,
  'match': 'Brentford vs Liverpool',
  'profit_margin_%': 40.869010942313786},
 {'combination': {'Brighton and Hove Albion': (2.06, 'Matchbook'),
                  'Draw': (4.1, 'Betfair'),
                  'Leeds United': (5.1, 'Smarkets')},
  'h_val': 0.9254177636008227,
  'match': 'Brighton and Hove Albion vs Leeds United',
  'profit_margin_%': 8.059304600873007},
 {'combination': {'Arsenal': (1.43, 'Matchbook'),
                  'Burnley': (12.5, 'Smarkets'),
                  'Draw': (6.2, 'Betfair')},
  'h_val': 0.9405910218813445,
  'match': 'Burnley vs Arsenal',
  'profit_margin_%': 6.316132807628483}]
